## 1. Importar Librerías

In [ ]:
import asyncio
import os
from dotenv import load_dotenv
import random
from datetime import datetime
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from azure.identity import DefaultAzureCredential
from azure.identity.aio import get_bearer_token_provider
from pydantic import Field

## 2. Configuración

In [ ]:
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
MODEL = os.getenv("OPENAI_MODEL") or os.getenv("MODEL")

if not AZURE_OPENAI_ENDPOINT or not MODEL:
    raise RuntimeError("Faltan variables de entorno: AZURE_OPENAI_ENDPOINT y/o OPENAI_MODEL")

print(f"✅ Configuración cargada: {MODEL}")

## 3. Crear Cliente OpenAI

In [ ]:
client = OpenAIChatClient(
    base_url=AZURE_OPENAI_ENDPOINT.rstrip("/") + "/openai/v1/",
    api_key=get_bearer_token_provider(
        DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
    ),
    model_id=MODEL,
)

## 4. Herramientas para Weekend Agent

Estas herramientas ayudarán al agente de fin de semana a obtener información relevante.

In [ ]:
def get_weather(
    city: Annotated[str, Field(description="The city to get the weather for.")],
    date: Annotated[str, Field(description="The date to get weather for in format YYYY-MM-DD.")],
) -> dict:
    """Simula obtener el clima."""
    print(f"  🌤️  Getting weather for {city} on {date}")
    if random.random() < 0.05:
        return {"temperature": 72, "description": "Sunny"}
    else:
        return {"temperature": 60, "description": "Rainy"}

def get_activities(
    city: Annotated[str, Field(description="The city to get activities for.")],
    date: Annotated[str, Field(description="The date to get activities for in format YYYY-MM-DD.")],
) -> list[dict]:
    """Simula obtener actividades disponibles."""
    print(f"  🎯 Getting activities for {city} on {date}")
    return [
        {"name": "Hiking", "location": city},
        {"name": "Beach", "location": city},
        {"name": "Museum", "location": city},
    ]

def get_current_date() -> str:
    """Obtiene la fecha actual."""
    print("  📅 Getting current date")
    return datetime.now().strftime("%Y-%m-%d")

print("✅ Herramientas de Weekend Agent definidas")

## 5. Herramientas para Meal Agent

Estas herramientas ayudan al agente de comidas.

In [ ]:
def find_recipes(
    query: Annotated[str, Field(description="User query or desired meal/ingredient")],
) -> list[dict]:
    """Busca recetas basadas en una consulta."""
    print(f"  🍳 Finding recipes for '{query}'")
    if "pasta" in query.lower():
        recipes = [
            {
                "title": "Pasta Primavera",
                "ingredients": ["pasta", "vegetables", "olive oil"],
                "steps": ["Cook pasta.", "Sauté vegetables."],
            }
        ]
    elif "tofu" in query.lower():
        recipes = [
            {
                "title": "Tofu Stir Fry",
                "ingredients": ["tofu", "soy sauce", "vegetables"],
                "steps": ["Cube tofu.", "Stir fry veggies."],
            }
        ]
    else:
        recipes = [
            {
                "title": "Grilled Cheese Sandwich",
                "ingredients": ["bread", "cheese", "butter"],
                "steps": ["Butter bread.", "Place cheese between slices.", "Grill until golden brown."],
            }
        ]
    return recipes

def check_fridge() -> list[str]:
    """Simula revisar ingredientes en el refrigerador."""
    print("  🥗 Checking fridge for current ingredients")
    if random.random() < 0.5:
        items = ["pasta", "tomato sauce", "bell peppers", "olive oil"]
    else:
        items = ["tofu", "soy sauce", "broccoli", "carrots"]
    return items

print("✅ Herramientas de Meal Agent definidas")

## 6. Crear Agentes Especializados

Creamos los dos agentes especializados con sus respectivas herramientas e instrucciones.

In [ ]:
# Agente para planificación de fin de semana
weekend_agent = ChatAgent(
    chat_client=client,
    instructions=(
        "You help users plan their weekends and choose the best activities for the given weather. "
        "If an activity would be unpleasant in the weather, don't suggest it. "
        "Include the date of the weekend in your response."
    ),
    tools=[get_weather, get_activities, get_current_date],
)

# Agente para planificación de comidas
meal_agent = ChatAgent(
    chat_client=client,
    instructions=(
        "You help users plan meals and choose the best recipes. "
        "Include the ingredients and cooking instructions in your response. "
        "Indicate what the user needs to buy from the store when their fridge is missing ingredients."
    ),
    tools=[find_recipes, check_fridge],
)

print("✅ Agentes especializados creados")
print("   - Weekend Agent: planificación de actividades")
print("   - Meal Agent: planificación de comidas")

## 7. Crear Funciones Wrapper

Estas funciones permiten al supervisor invocar a los agentes especializados como si fueran herramientas.

In [ ]:
async def plan_weekend(query: str) -> str:
    """Invoca al agente de fin de semana."""
    print("\n🔧 Tool: plan_weekend invoked")
    response = await weekend_agent.run(query)
    return response.text

async def plan_meal(query: str) -> str:
    """Invoca al agente de comidas."""
    print("\n🔧 Tool: plan_meal invoked")
    response = await meal_agent.run(query)
    return response.text

print("✅ Funciones wrapper creadas")

## 8. Crear el Agente Supervisor

El supervisor es el cerebro del sistema. Analiza la consulta del usuario y decide:
- ¿Necesito al weekend agent?
- ¿Necesito al meal agent?
- ¿Necesito a ambos?

Luego sintetiza las respuestas en una respuesta coherente final.

In [ ]:
supervisor_agent = ChatAgent(
    chat_client=client,
    instructions=(
        "You are a supervisor managing two specialist agents: a weekend planning agent and a meal planning agent. "
        "Break down the user's request, decide which specialist (or both) to call via the available tools, "
        "and then synthesize a final helpful answer. When invoking a tool, provide clear, concise queries."
    ),
    tools=[plan_weekend, plan_meal],
)

print("✅ Supervisor Agent creado")
print("   El supervisor puede invocar:")
print("   - plan_weekend()")
print("   - plan_meal()")

## 9. Ejecutar Consulta Combinada

Probemos con una consulta que requiere ambos agentes.

In [ ]:
async def main():
    print("\n" + "="*80)
    print("CONSULTA AL SUPERVISOR")
    print("="*80)
    
    user_query = "What can I do this weekend and what can I cook for lunch?"
    print(f"\n👤 Usuario: {user_query}\n")
    print("─" * 80)
    
    response = await supervisor_agent.run(user_query)
    
    print("\n" + "─" * 80)
    print("\n🤖 RESPUESTA FINAL DEL SUPERVISOR:\n")
    print(response.text)
    print("\n" + "="*80)

await main()

## 10. Probar Solo Fin de Semana

In [ ]:
async def test_weekend_only():
    print("\n" + "="*80)
    print("PRUEBA: SOLO PLANIFICACIÓN DE FIN DE SEMANA")
    print("="*80 + "\n")
    
    query = "What activities would be good for this weekend in Seattle?"
    print(f"👤 Usuario: {query}\n")
    print("─" * 80)
    
    response = await supervisor_agent.run(query)
    
    print("\n" + "─" * 80)
    print("\n🤖 Respuesta:\n")
    print(response.text)

await test_weekend_only()

## 11. Probar Solo Comidas

In [ ]:
async def test_meal_only():
    print("\n" + "="*80)
    print("PRUEBA: SOLO PLANIFICACIÓN DE COMIDAS")
    print("="*80 + "\n")
    
    query = "What should I cook for dinner tonight? I like pasta."
    print(f"👤 Usuario: {query}\n")
    print("─" * 80)
    
    response = await supervisor_agent.run(query)
    
    print("\n" + "─" * 80)
    print("\n🤖 Respuesta:\n")
    print(response.text)

await test_meal_only()

## Conclusión

Este patrón de **Supervisor Agent** demuestra:

### 1. **Arquitectura Multi-Agente**
- Especialización: cada agente tiene su dominio
- Coordinación: el supervisor orquesta
- Escalabilidad: fácil agregar más agentes

### 2. **Routing Inteligente**
- El supervisor analiza la consulta
- Decide qué agentes invocar
- Puede llamar a múltiples agentes en secuencia

### 3. **Composición de Resultados**
- Recopila respuestas de múltiples agentes
- Sintetiza información coherente
- Proporciona respuesta unificada al usuario

### Ventajas del Patrón:
- **Modularidad**: Cada agente es independiente
- **Mantenibilidad**: Fácil actualizar agentes individuales
- **Reutilización**: Agentes pueden usarse solos o coordinados
- **Especialización**: Cada agente tiene su contexto y herramientas

### Aplicaciones Prácticas:
- Sistemas de atención al cliente (agentes por departamento)
- Asistentes personales completos
- Automatización empresarial (finanzas, HR, IT)
- Plataformas de gestión integrada

Este patrón es especialmente útil cuando tienes dominios claramente separados y cada uno requiere herramientas o conocimientos específicos.